### SIR Model

In [ ]:
import numpy as np
from scipy.integrate import odeint
from ipywidgets import *
from bqplot import *

In [ ]:
def SIR_model(S0, I0, R0, beta, gamma, mu, N0, T):
    t = np.arange(0, T + 0.1, 0.1)

    def deriv(y, t, beta, gamma, mu):
        S, I, R = y
        dSdt = -beta * S * I / N0
        dIdt = beta * S * I / N0 - gamma * I - mu * I
        dRdt = gamma * I

        return dSdt, dIdt, dRdt

    y0 = S0, I0, R0
    S, I, R = odeint(deriv, y0, t, args=(beta, gamma, mu)).T
    return S, I, R, N0 - np.sum([S, I, R], axis=0)

In [ ]:
N0 = 100
T = 50
I0 = 1
R0 = 0
D0 = 0
S0 = N0 - I0 - R0 - D0
beta = 0.5
gamma = 0.2
mu = 0.01
S, I, R, D = SIR_model(S0, I0, R0, beta, gamma, mu, N0, T)

In [ ]:
style = {"description_width": "initial"}
I0_slider = IntSlider(
    description="#Infected (at t=0)", value=I0, min=0, max=N0, step=1, style=style
)
beta_slider = FloatSlider(
    description="&beta;", value=beta, min=0, max=10.0, step=0.01, style=style
)
mu_slider = FloatSlider(
    description="&mu;", value=mu, min=0, max=0.5, step=0.01, style=style
)
gamma_slider = FloatSlider(
    description="&gamma;", value=gamma, min=0, max=10.0, step=0.01, style=style
)

In [ ]:
title = r"SIR dynamic with N = 100, Beta = {0:0.2f}, Gamma = {1:0.2f}, Mu = {2:0.2f}, R0 = {3:0.2f}"
title2 = r"SIR dynamic with N = 100, Beta = {0:0.2f}, Gamma = {1:0.2f}, Mu = {2:0.2f}, R0 = +infinity"
sc_y = LinearScale(min=0, max=100, label="Days")
sc_t = LinearScale(min=0, max=T, label="Number")
ax_t = Axis(scale=sc_t, label="Days", grid_lines="none")
ax_y = Axis(scale=sc_y, label="Number", orientation="vertical")
fig_model = Figure(
    title=title.format(beta, gamma, mu, beta / (gamma + mu)),
    animation_duration=1000,
    axes=[ax_t, ax_y],
)
mark_mod = Lines(
    labels=[
        "Susceptible",
        "Infected",
        "Recovered",
        "Deaths",
        "Cumulative Nbr of Infected",
    ],
    colors=["gray", "orange", "green", "red", "dodgerblue"],
    display_legend=True,
    stroke_width=3,
    scales={"x": sc_t, "y": sc_y},
)
mark_mod.x = np.arange(0, T + 0.1, 0.1)
mark_mod.y = [S, I, R, D, N0 - S]
fig_model.marks = [mark_mod]
fig_model.layout.width = "700px"
fig_model.layout.height = "700px"

In [ ]:
table_tmpl_SIR = \
    """
<table width="400px">
<caption align="center">Parameters</caption>
<tr>
    <td style="font-weight: bold", width ="350px">Initial Population</td>
    <td align="right", width ="50px">{0:d}</td>
</tr>
<tr>
    <td style="font-weight: bold">Initial number of cases</td>
    <td align="right">{1:d}</td>
</tr>
<tr>
    <td style="font-weight: bold">Initial number of recovered</td>
    <td align="right">{2:d}</td>
</tr>
<tr>
    <td style="font-weight: bold">Initial number of deaths</td>
    <td align="right">{3:d}</td>
</tr>
<tr>
    <td style="font-weight: bold"> (&beta;) Infection Rate</td>
    <td align="right">{4:.2f}</td>
</tr>
<tr>
    <td style="font-weight: bold"> (&gamma;) Recovery Rate</td>
    <td align="right">{5:.2f}</td>
</tr>
<tr>
    <td style="font-weight: bold"> (&mu;) Death Rate</td>
    <td align="right">{6:.2f}</td>
</tr>
</table>
"""

stats_table1 = HTML()
stats_table1.value = table_tmpl_SIR.format(N0, I0, R0, D0, beta, gamma, mu)
stats_table1.layout.width = '600px'

In [ ]:
def update_mod(*args):
    global N0, D0, R0, T
    I0 = I0_slider.value
    S0 = N0 - I0 - R0 - D0
    beta = beta_slider.value
    mu = mu_slider.value
    gamma = gamma_slider.value
    res = SIR_model(S0, I0, R0, beta, gamma, mu, N0, T)
    stats_table1.value = table_tmpl_SIR.format(N0, I0, R0, D0, beta, gamma, mu)
    mark_mod.y = [res[0], res[1], res[2], res[3], N0 - res[0]]
    if gamma + mu < 1e-10:
        fig_model.title = title2.format(beta, gamma, mu)
    else:
        fig_model.title = title.format(beta, gamma, mu, beta / (gamma + mu))


I0_slider.observe(update_mod, "value")
I0_slider.continuous_update = False
I0_slider.layout.width = "400px"
beta_slider.observe(update_mod, "value")
beta_slider.continuous_update = False
beta_slider.layout.width = "400px"
mu_slider.observe(update_mod, "value")
mu_slider.continuous_update = False
mu_slider.layout.width = "400px"
gamma_slider.observe(update_mod, "value")
gamma_slider.continuous_update = False
gamma_slider.layout.width = "400px"

Deterministic SIR model with 4 different states :
- Susceptible (not infected by the virus)
- Infected
- Recovered or cured
- Dead

SIR model is described by a system of ODEs which gives the number of people at time t in each state.

$\frac{dS}{dt} = -\frac{\beta S I}{N}$

$\frac{dI}{dt} = \frac{\beta S I}{N} - (\mu + \gamma) I$

$\frac{dR}{dt} = \gamma I$

$\frac{dD}{dt} = \mu I$

- $\beta$ is the transition rate from S to I
- $\gamma$ is the recovery rate or the transition rate from I to R
- $\mu$ is the death rate or the transition rate from I to D

We suppose here that the population N is constant.

$R_0$ of an infection can be thought of as the expected number of cases directly generated by one case in a population where all individuals are susceptible to infection.

In this model $R_0 = \frac{\beta}{\mu + \gamma}$. The maximum of number of infected $I_{max}$ and the final number of infected $N-S_{\infty}$ depends only $R_0$ and $N$. If $R_0>1$ a virus outbreak will occur.

When it exists $I_{max}=S_0+I_0 - \frac{N}{R_0} (1 -\log{\frac{N}{S_0 R_0}}) \approx N - \frac{N}{R_0}(1 +\log{R_0})$

$S_{\infty}$ is the unique root in $]0,  \frac{N}{R_0}[$ of the equation :
$0=N - S_{\infty} + \frac{N}{R_0} \log{\frac{S_{\infty}}{N-1}}$

In [ ]:
HBox([fig_model, VBox([I0_slider, beta_slider, gamma_slider, mu_slider, stats_table1])])